# Download and load to BQ - IMBD

This dataset from [Kaggle.com](https://www.kaggle.com/ashirwadsangwan/imdb-dataset) is a fairly rich IMBD rating dataset
We will use this dataset to develop a pipeline that utilizes an advanced cross-two-tower network impleneted with Tensorflow 2.6
Next notebooks will perform data prep and create the model

In [2]:
from google.cloud import bigquery
import os

In [3]:
BUCKET = "vertex-stuff_data_transfers"
BQ_DATASET = 'imdb'
PROJECT_ID = 'vertex-stuff'

## Connect to common bucket
Folder structure

```/
  /data
    /unzipped
  /embeddings
  /logs
```

#### Data Provenance:

* https://datasets.imdbws.com/title.ratings.tsv.gz
* https://datasets.imdbws.com/name.basics.tsv.gz
* https://datasets.imdbws.com/title.principals.tsv.gz
* https://datasets.imdbws.com/title.akas.tsv.gz
* https://datasets.imdbws.com/title.basics.tsv.gz

PUT THESE FILES IN `{BUCKET}/data`

Missing values are denoted by `\N` per the uploader's notes so we will remove for easier handling by BQ

In [4]:
# unzip the files

# get the movielens 25 mil dataset
!wget https://files.grouplens.org/datasets/movielens/ml-25m.zip
# save to cloud storage
!gsutil cp ml-25m.zip gs://$BUCKET/data


tables = ['title.ratings', 'name.basics', 'title.principals', 'title.akas', 'title.basics']

# Unzip each file

def unzip(table):
    command = f"gsutil cat gs://{BUCKET}/data/{table}.tsv.gz | zcat | tr -d '\\\\N(?!$)' |  gsutil cp - gs://{BUCKET}/data/unzipped/{table}.tsv"
    os.system(command)
    
for t in tables:
    unzip(t)

--2021-10-29 14:11:22--  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip.4’

ml-25m.zip.4        100%[===================>] 249.84M  6.17MB/s    in 40s     

utime(ml-25m.zip.4): No such file or directory
2021-10-29 14:12:02 (6.32 MB/s) - ‘ml-25m.zip.4’ saved [261978986/261978986]

CommandException: No URLs matched: ml-25m.zip


Copying from <STDIN>...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              
Copying from <STDIN>...
/ [1 files][    0.0 B/    0.0 B]      0.0 B/s                                   
Operation completed over 1 objects.                                              
Copying from <STDIN>...
/ [1 files][    0.0 B/    0.0 B]      0.0 B/s                                   
Operation completed over 1 objects.                                              
Copying from <STDIN>...
/ [1 files][    0.0 B/    0.0 B]      0.0 B/s                                   
Operation completed over 1 objects.                                              
Copying from <STDIN>...
/ [1 files][    0.0 B/    0.0 B]      0.0 B/s                                   
Operation completed over 1 objects.                                              


## Movielens data

https://cloud.google.com/bigquery-ml/docs/bigqueryml-mf-explicit-tutorial

In [12]:
%%bash
curl -O 'http://files.grouplens.org/datasets/movielens/ml-1m.zip'
unzip ml-1m.zip
bq mk --dataset movielens
sed 's/::/,/g' ml-1m/ratings.dat > ratings.csv
bq load --source_format=CSV movielens.movielens_1m ratings.csv \
  user_id:INT64,item_id:INT64,rating:FLOAT64,timestamp:TIMESTAMP

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
BigQuery error in mk operation: Dataset 'vertex-stuff:movielens' already exists.


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5778k  100 5778k    0     0  18.2M      0 --:--:-- --:--:-- --:--:-- 18.2M
Upload complete.
Waiting on bqjob_r580039f3b9903eff_0000017ccc5d6aa5_1 ... (7s) Current status: DONE   


In [5]:
%%bash
sed 's/::/@/g' ml-1m/movies.dat > movie_titles.csv
bq load --source_format=CSV --field_delimiter=@ \
 movielens.movie_titles movie_titles.csv \
 movie_id:INT64,movie_title:STRING,genre:STRING

Upload complete.
Waiting on bqjob_r6fff72891793a2b1_0000017ccc6637e1_1 ... (3s) Current status: DONE   


### Only run once below to create BQ Dataset

In [6]:
# Construct a BigQuery client object.
client = bigquery.Client()

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(f'{PROJECT_ID}.{BQ_DATASET}')

dataset = client.create_dataset(dataset, timeout=30)

In [ ]:
#Create landing Tables

def load_table(table_id):
    client = bigquery.Client()
    uri = f'gs://{BUCKET}/data/unzipped/{table_id}.tsv'
    job_config = bigquery.LoadJobConfig(
        autodetect=True,
        skip_leading_rows=1,
        source_format=bigquery.SourceFormat.CSV,
        field_delimiter='\t',
        max_bad_records = 10000,
        #null_marker = '||'
    )
    table_id = table_id.replace('.', '_')
    fqtn = f'{BQ_DATASET}.{table_id}'
    load_job = client.load_table_from_uri(
        uri, destination=fqtn, job_config=job_config
    )
    
    load_job.result()
    
    destination_table = client.get_table(fqtn)
    return(print("Loaded {} rows from {}.".format(destination_table.num_rows, uri)))

for t in tables:
    load_table(t)

Loaded 1197953 rows from gs://vertex-stuff_data_transfers/data/unzipped/title.ratings.tsv.


## Create the user and movie views

In [ ]:
%%bigquery

CREATE OR REPLACE TABLE `matching-engine-blog.movielens.user_view` AS
SELECT 
	CAST(user_id as STRING) as user_id,
    reviews.item_id as movie_id,
    rating,
    timestamp, 
    EXTRACT(WEEK FROM timestamp) as week_num,
    EXTRACT(HOUR FROM timestamp) as hour,
FROM 
    `matching-engine-blog.movielens.movie_titles` mt,
    `matching-engine-blog.movielens.movielens_1m` reviews,
    `matching-engine-blog.imdb.title_basics` tb 
WHERE 
    mt.movie_id = reviews.item_id AND 
    UPPER(CONCAT(tb.originalTitle, " (", tb.startYear, ")")) = UPPER(mt.movie_title)
;

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `matching-engine-blog.movielens.movie_view` AS
WITH 
    actors AS (
        SELECT 
        tp.tconst,
        STRING_AGG(nb.nconst, ' ') actors,
        STRING_AGG(REPLACE(nb.primaryProfession, ",", " "), ' ') as profession
        FROM 
        `matching-engine-blog.imdb.title_principals` tp,
        `matching-engine-blog.imdb.name_basics` nb
        WHERE
        tp.nconst = nb.nconst AND 
        tp.category = "actor"
        group by tp.tconst
    ), 
    directors AS (
        SELECT 
        tp.tconst,
        STRING_AGG(nb.nconst, ' ') actors,
        STRING_AGG(REPLACE(nb.primaryProfession, ",", " "), ' ') as profession
        FROM 
        `matching-engine-blog.imdb.title_principals` tp,
        `matching-engine-blog.imdb.name_basics` nb
        WHERE
        tp.nconst = nb.nconst AND 
        tp.category = "director"
        group by tp.tconst
  )
SELECT 
    cast(mt.movie_id as STRING) as movie_id,
    tb.originalTitle as title,
    tb.isAdult as is_adult,
    #tb.startYear as start_year,
    #tb.runtimeMinutes,
    #REPLACE(tb.genres, ',', ' ') as genres,
    tr.averageRating as average_rating,
    tr.numVotes as num_votes,
    directors.actors as director,
    directors.profession as d_profession,
    actors.actors,
    actors.profession as actor_profession

FROM
    `matching-engine-blog.imdb.title_basics` tb ,
    `matching-engine-blog.imdb.title_ratings` tr ,
    `matching-engine-blog.movielens.movie_titles` mt,
    directors,
    actors
WHERE 
    directors.tconst = tb.tconst AND
    actors.tconst = tb.tconst AND
    tb.tconst = tr.tconst AND 
    UPPER(CONCAT(tb.originalTitle, " (", tb.startYear, ")")) = UPPER(mt.movie_title)